**Analiza i predikcija srčanog udara - (Kaggle dataset)**


| **Značajka** | **Opis**                                                                                                                                                                                                                                                                              |
|--------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| age          | Dob u godinama                                                                                                                                                                                                                                                                         |
| sex          | Spol: 1 = muško, 0 = žensko                                                                                                                                                                                                                                                                    |
| cp           | Vrsta bolova u prsima: <br> 0: Tipična angina - bol u prsima povezana smanjenjem dotoka krvi u srce <br> 1: Atipična angina - bol u prsima koja nije povezana s srcem <br> 2: Neanginalna bol - obično ezofagealni grčevi (ne povezani s srcem) <br> 3: Asimptomatski - bol u prsima bez znakova bolesti |
| trestbps     | Krvni tlak u mirovanju (u mm Hg prilikom prijema u bolnicu). Sve iznad 130-140 obično je razlog za zabrinutost                                                                                                                                                                     |
| chol         | Serumski kolesterol u mg/dl. Serum = LDL + HDL + .2 * trigliceridi. Vrijednosti iznad 200 uzrokuju zabrinutost                                                                                                                                                                       |
| fbs          | Razina glukoze u krvi natašte > 120 mg/dl (1 = istina, 0 = laž)                                                                                                                                                                                                                         |
| restecg      | Rezultati elektrokardiografskog snimanja u mirovanju: <br> 0: Ništa posebno <br> 1: Abnormalnost ST-T valova - signalizira nepravilan otkucaj srca <br> 2: Moguća ili definitivna hipertrofija lijeve klijetke - proširenje glavnog kamere srca za pumpanje                                                                   |
| thalach      | Maksimalna postignuta brzina otkucaja srca                                                                                                                                                                                                                                                              |
| exang        | Angina uzrokovana vježbanjem (1 = da, 0 = ne)                                                                                                                                                                                                                                                          |
| oldpeak      | ST depresija uzrokovana vježbanjem u odnosu na mirovanje. Pokazuje stres srca tijekom vježbanja. Nezdravo srce će se više stresirati                                                                                                                                                         |
| slope        | Nagib vrha vježbanja segmenta ST: <br> 0: Uzlazni - bolja brzina otkucaja srca uz vježbanje (neobično) <br> 1: Ravni - minimalna promjena (tipično zdravo srce) <br> 2: Silazni - znakovi nezdravog srca                                                                  |
| ca           | Broj glavnih krvnih žila (0-3) koje su obojene fluoroskopijom. Obojena žila znači da liječnik može vidjeti krv koja prolazi kroz nju. Što više krvnog pokreta, to bolje (nema ugrušaka)                                                                                                            |
| thal         | Rezultat talijanskog stresnog testa: <br> 1, 3: Normalno <br> 6: Fiksni defekt - nekad je bio defekt, ali je sada u redu <br> 7: Obratni defekt - nema pravilnog krvnog toka tijekom vježbanja                                                                                                                      |
| target       | Ima li bolest ili ne (1 = da, 0 = ne)                                                                                                                                                                                                                                                                               |


In [92]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [93]:
file_path = r'C:\Users\HP PC\Desktop\heart.csv'
data = pd.read_csv(file_path)

print(data.head())

   age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
0   63    1   3     145   233    1        0       150     0      2.3    0   
1   37    1   2     130   250    0        1       187     0      3.5    0   
2   41    0   1     130   204    0        0       172     0      1.4    2   
3   56    1   1     120   236    0        1       178     0      0.8    2   
4   57    0   0     120   354    0        1       163     1      0.6    2   

   caa  thall  output  
0    0      1       1  
1    0      2       1  
2    0      2       1  
3    0      2       1  
4    0      2       1  


In [94]:
print(data.info())
print(data.isnull().sum())


X = data.iloc[:, :-1].values #ulazne
y = data.iloc[:, -1].values #ciljna 


print(y.dtype)
print(set(y))


print(y.dtype)
print(set(y))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB
None
age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64
int64

In [95]:
# Normalizacija podataka
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [96]:
#Treniranje i vrednovanje modela neuronske mreže korištenjem PyTorcha

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)


dataset = TensorDataset(X_tensor, y_tensor)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

input_size = X.shape[1]
hidden_size = 50
num_classes = len(set(y))
model = NeuralNetwork(input_size, hidden_size, num_classes)

# funkcija gubitka
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treniranje modela
num_epochs = 20
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Vrednovanje modela
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test set: {accuracy:.2f}%')

Epoch [1/20], Loss: 0.7640
Epoch [2/20], Loss: 0.6219
Epoch [3/20], Loss: 0.6285
Epoch [4/20], Loss: 0.6058
Epoch [5/20], Loss: 0.5093
Epoch [6/20], Loss: 0.5211
Epoch [7/20], Loss: 0.3900
Epoch [8/20], Loss: 0.3884
Epoch [9/20], Loss: 0.2842
Epoch [10/20], Loss: 0.3552
Epoch [11/20], Loss: 0.2597
Epoch [12/20], Loss: 0.2864
Epoch [13/20], Loss: 0.3724
Epoch [14/20], Loss: 0.2714
Epoch [15/20], Loss: 0.3248
Epoch [16/20], Loss: 0.2138
Epoch [17/20], Loss: 0.5033
Epoch [18/20], Loss: 0.1762
Epoch [19/20], Loss: 0.3494
Epoch [20/20], Loss: 0.1869
Accuracy of the model on the test set: 78.69%


In [97]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistička regresija
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
log_reg_acc = accuracy_score(y_test, y_pred_log_reg)

# KNN
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
knn_acc = accuracy_score(y_test, y_pred_knn)

# SupportVectorMachine
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)

# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)

# Stablo odluke
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
dt_acc = accuracy_score(y_test, y_pred_dt)


print(f'Accuracy of Neural Network: {accuracy:.2f}%')
print(f'Accuracy of Logistic Regression: {log_reg_acc:.2f}%')
print(f'Accuracy of KNN: {knn_acc:.2f}%')
print(f'Accuracy of SVM: {svm_acc:.2f}%')
print(f'Accuracy of Random Forest: {rf_acc:.2f}%')
print(f'Accuracy of Decision Tree: {dt_acc:.2f}%')

Accuracy of Neural Network: 78.69%
Accuracy of Logistic Regression: 0.85%
Accuracy of KNN: 0.93%
Accuracy of SVM: 0.87%
Accuracy of Random Forest: 0.87%
Accuracy of Decision Tree: 0.82%


In [99]:
results = {
    'Model': ['Logistic Regression', 'K-nearest neighbors', 'Support Vector Machine', 'Random Forest', 'Decision Tree', 'Neural Network'],
    'Testing Accuracy %': [log_reg_acc * 100, knn_acc * 100, svm_acc * 100, rf_acc * 100, dt_acc * 100, accuracy]
}

results_df = pd.DataFrame(results)


print(results_df)


                    Model  Testing Accuracy %
0     Logistic Regression           85.245902
1     K-nearest neighbors           93.442623
2  Support Vector Machine           86.885246
3           Random Forest           86.885246
4           Decision Tree           81.967213
5          Neural Network           78.688525
